<a href="https://colab.research.google.com/github/Baglecake/blank-app/blob/main/Series_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries
!pip install qiskit optuna seaborn matplotlib

# Import necessary libraries
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity, entropy
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, amplitude_damping_error, phase_damping_error
import optuna
import pandas as pd
import seaborn as sns
import itertools
import warnings
import logging

# Configure logging for better debug messages
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Suppress deprecation warnings for cleaner output
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Set a random seed for reproducibility
np.random.seed(42)

class SimulationParameters:
    """
    Holds all simulation-related parameters.
    """
    def __init__(self, steps=500, num_qubits=4,
                 sinusoidal_scale=5.0, harmonic_scale=3.0,
                 cubic_scale=0.9, exponential_scale=5.0,
                 antifinity_factor=0.01, distinction_epsilon=0.0001,
                 recursion_depth=200, feedback_scale=1.5,
                 memory_decay=0.5, decoherence_rate=0.01,
                 h_field=1.0, j_coupling=0.5,
                 max_surplus_angle=np.pi / 2,
                 distinction_variance_reduction=0.1,
                 surplus_distribution_std=0.1,
                 noise_model=None,
                 basis_gates=None,
                 coupling_map=None,
                 measurement_error=None,
                 enable_plotting=False):  # New parameter
        self.steps = steps
        self.num_qubits = num_qubits
        self.sinusoidal_scale = sinusoidal_scale
        self.harmonic_scale = harmonic_scale
        self.cubic_scale = cubic_scale
        self.exponential_scale = exponential_scale
        self.antifinity_factor = antifinity_factor
        self.distinction_epsilon = distinction_epsilon
        self.recursion_depth = recursion_depth
        self.feedback_scale = feedback_scale
        self.memory_decay = memory_decay
        self.decoherence_rate = decoherence_rate
        self.h_field = h_field
        self.j_coupling = j_coupling
        self.max_surplus_angle = max_surplus_angle
        self.distinction_variance_reduction = distinction_variance_reduction
        self.surplus_distribution_std = surplus_distribution_std
        self.noise_model = noise_model
        self.basis_gates = basis_gates
        self.coupling_map = coupling_map
        self.measurement_error = measurement_error
        self.enable_plotting = enable_plotting  # Initialize the plotting flag

    def __repr__(self):
        return (f"SimulationParameters(steps={self.steps}, num_qubits={self.num_qubits}, "
                f"sinusoidal_scale={self.sinusoidal_scale}, harmonic_scale={self.harmonic_scale}, "
                f"cubic_scale={self.cubic_scale}, exponential_scale={self.exponential_scale}, "
                f"antifinity_factor={self.antifinity_factor}, distinction_epsilon={self.distinction_epsilon}, "
                f"recursion_depth={self.recursion_depth}, feedback_scale={self.feedback_scale}, "
                f"memory_decay={self.memory_decay}, decoherence_rate={self.decoherence_rate}, "
                f"h_field={self.h_field}, j_coupling={self.j_coupling}, "
                f"max_surplus_angle={self.max_surplus_angle}, "
                f"distinction_variance_reduction={self.distinction_variance_reduction}, "
                f"surplus_distribution_std={self.surplus_distribution_std}, "
                f"enable_plotting={self.enable_plotting})")

class MutualInformation:
    """
    Calculates and tracks mutual information between qubits.
    """
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.mutual_info_tracker = []

    def calculate_mutual_information(self, density_matrix, step):
        if not isinstance(density_matrix, DensityMatrix):
            raise TypeError("density_matrix must be an instance of Qiskit's DensityMatrix.")

        mi_matrix = np.zeros((self.num_qubits, self.num_qubits))

        for i in range(self.num_qubits):
            for j in range(i+1, self.num_qubits):
                # Calculate reduced density matrices
                rho_i = partial_trace(density_matrix, [q for q in range(self.num_qubits) if q != i])
                rho_j = partial_trace(density_matrix, [q for q in range(self.num_qubits) if q != j])
                rho_ij = partial_trace(density_matrix, [q for q in range(self.num_qubits) if q not in [i, j]])

                if not isinstance(rho_i, DensityMatrix) or not isinstance(rho_j, DensityMatrix) or not isinstance(rho_ij, DensityMatrix):
                    raise TypeError("Reduced density matrices must be instances of Qiskit's DensityMatrix.")

                # Calculate entropies
                S_i = entropy(rho_i)
                S_j = entropy(rho_j)
                S_ij = entropy(rho_ij)

                # Calculate mutual information
                mi = S_i + S_j - S_ij
                mi_matrix[i, j] = mi
                mi_matrix[j, i] = mi  # Mutual information is symmetric

        self.mutual_info_tracker.append(mi_matrix)
        return mi_matrix

    def get_average_mutual_information(self):
        if not self.mutual_info_tracker:
            return None
        return np.mean(self.mutual_info_tracker, axis=0)

class QuantumSystem:
    """
    Manages the quantum system's state and applies various quantum operations.
    """
    def __init__(self, params):
        self.params = params
        self.tracker = np.zeros((params.num_qubits, params.steps))
        self.feedback_memory = np.zeros(params.num_qubits)
        self.distinction_tracker = np.zeros((params.num_qubits, params.steps))
        self.antifinity_tracker = np.zeros(params.steps)
        self.recursion_tracker = np.zeros((params.num_qubits, params.steps))
        self.density_matrix = None  # Initialize density_matrix
        self.initial_state = None

    def apply_surplus(self, step, qubit):
        sinusoidal_surplus = self.params.sinusoidal_scale * np.sin(2 * np.pi * step / self.params.steps + qubit * 0.1)
        harmonic_surplus = self.params.harmonic_scale * np.sin(step * np.pi / 15)
        cubic_term = self.params.cubic_scale * (sinusoidal_surplus ** 3)
        exponential_term = self.params.exponential_scale * np.exp(harmonic_surplus)

        base_surplus = sinusoidal_surplus + harmonic_surplus + cubic_term + exponential_term

        feedback = self.params.feedback_scale * self.feedback_memory[qubit]
        total_surplus = base_surplus + feedback

        # Limit surplus magnitude
        total_surplus = np.clip(total_surplus, -self.params.max_surplus_angle, self.params.max_surplus_angle)

        self.tracker[qubit, step] = total_surplus
        self.feedback_memory[qubit] = (self.params.memory_decay * self.feedback_memory[qubit] +
                                        (1 - self.params.memory_decay) * total_surplus)

        if np.abs(total_surplus) > self.params.distinction_epsilon:
            self.trigger_distinction(step, qubit)

        return total_surplus

    def trigger_distinction(self, step, qubit):
        window_start = max(0, step - 10)
        mean = self.tracker[qubit, window_start:step].mean()
        std = self.tracker[qubit, window_start:step].std()
        # Reduce distinction variance
        std *= self.params.distinction_variance_reduction
        distinction_value = np.random.normal(mean, std)
        self.distinction_tracker[qubit, step] = distinction_value

        # Record the distinction value for recursive application
        self.apply_recursion(step, qubit, distinction_value)

    def apply_recursion(self, step, qubit, distinction_value):
        for depth in range(self.params.recursion_depth):
            angle = distinction_value / (2 ** (depth + 1))
            self.recursion_tracker[qubit, step] += angle

    def apply_antifinity(self, step, circuit):
        decay = self.params.antifinity_factor * step
        self.antifinity_tracker[step] = decay
        for qubit in range(self.params.num_qubits):
            circuit.rx(decay, qubit)

    def apply_quantum_gate(self, step, qubit, circuit):
        total_surplus = self.apply_surplus(step, qubit)
        circuit.ry(total_surplus, qubit)

        if self.distinction_tracker[qubit, step] != 0:
            distinction_value = self.distinction_tracker[qubit, step]
            circuit.rz(distinction_value, qubit)

            for target_qubit in range(self.params.num_qubits):
                if target_qubit != qubit:
                    surplus_value = np.random.normal(
                        loc=distinction_value,
                        scale=self.params.surplus_distribution_std
                    )
                    circuit.cx(qubit, target_qubit)
                    circuit.rz(surplus_value, target_qubit)
                    circuit.cx(qubit, target_qubit)

            for depth in range(self.params.recursion_depth):
                angle = distinction_value / (2 ** (depth + 1))
                circuit.ry(angle, qubit)

class SimulationManager:
    """
    Orchestrates the simulation process, tracking metrics like entropy, coherence, purity, and fidelity.
    """
    def __init__(self, params):
        self.params = params
        self.system = QuantumSystem(params)
        self.entropy_tracker = np.zeros(params.steps)
        self.coherence_tracker = np.zeros(params.steps)
        self.purity_tracker = np.zeros(params.steps)
        self.fidelity_tracker = np.zeros(params.steps)
        self.relative_coherence_tracker = np.zeros(params.steps)
        self.mutual_info = MutualInformation(params.num_qubits)
        self.sv_simulator = AerSimulator(method='density_matrix', noise_model=params.noise_model, basis_gates=params.basis_gates)

    def run_simulation(self):
        initial_circuit = QuantumCircuit(self.params.num_qubits)
        for qubit in range(self.params.num_qubits):
            initial_circuit.h(qubit)
        initial_circuit.save_density_matrix(label='initial_state')
        result = self.sv_simulator.run(initial_circuit).result()

        if 'initial_state' not in result.data():
            raise ValueError("Initial density matrix 'initial_state' not found in simulation results.")

        self.system.density_matrix = DensityMatrix(result.data()['initial_state'])
        self.system.initial_state = self.system.density_matrix.copy()

        for step in range(1, self.params.steps):
            self.simulate_step(step)
            self.calculate_entropy(step)
            self.calculate_coherence(step)
            self.calculate_purity(step)
            self.calculate_fidelity(step)
            if step % 50 == 0:
                logger.info(f"Simulation progress: {step}/{self.params.steps} steps")

        if self.params.enable_plotting:
            self.mutual_info.plot_mutual_information()

    def simulate_step(self, step):
        step_circuit = QuantumCircuit(self.params.num_qubits)
        for qubit in range(self.params.num_qubits):
            self.system.apply_quantum_gate(step, qubit, step_circuit)
        self.system.apply_antifinity(step, step_circuit)

        step_circuit.save_density_matrix(label='state')
        circuit_to_transpile = QuantumCircuit(self.params.num_qubits)
        circuit_to_transpile.data = [
            instr for instr in step_circuit.data if instr.operation.name != 'save_density_matrix'
        ]

        transpiled_circuit = transpile(circuit_to_transpile, basis_gates=self.params.basis_gates, optimization_level=1)
        save_instrs = [instr for instr in step_circuit.data if instr.operation.name == 'save_density_matrix']
        for save_instr in save_instrs:
            transpiled_circuit.append(save_instr.operation, save_instr.qubits, save_instr.clbits)

        result = self.sv_simulator.run(transpiled_circuit, initial_state=self.system.density_matrix).result()

        if 'state' not in result.data():
            raise ValueError(f"Density matrix for step {step} not found in simulation results.")

        self.system.density_matrix = DensityMatrix(result.data()['state'])
        self.mutual_info.calculate_mutual_information(self.system.density_matrix, step)

    def calculate_entropy(self, step):
        density_matrix = self.system.density_matrix
        if not isinstance(density_matrix, DensityMatrix):
            raise TypeError(f"Density matrix at step {step} is not a DensityMatrix object.")

        entropies = []
        for qubit in range(self.params.num_qubits):
            reduced_dm = partial_trace(density_matrix, [q for q in range(self.params.num_qubits) if q != qubit])
            if not isinstance(reduced_dm, DensityMatrix):
                raise TypeError(f"Reduced density matrix for qubit {qubit} at step {step} is not a DensityMatrix object.")

            entropy_value = entropy(reduced_dm)
            entropies.append(entropy_value)

        self.entropy_tracker[step] = np.mean(entropies)

    def calculate_coherence(self, step):
        density_matrix = self.system.density_matrix
        if not isinstance(density_matrix, DensityMatrix):
            raise TypeError(f"Density matrix at step {step} is not a DensityMatrix object.")

        off_diagonal_sum = np.sum(np.abs(density_matrix.data)) - np.sum(np.abs(np.diag(density_matrix.data)))
        self.coherence_tracker[step] = off_diagonal_sum

        diag_dm = DensityMatrix(np.diag(np.diag(density_matrix.data)))
        rho = density_matrix
        rho_diag = diag_dm

        S_rho = entropy(rho)
        S_diag = entropy(rho_diag)

        C_rel = S_diag - S_rho
        self.relative_coherence_tracker[step] = C_rel

    def calculate_purity(self, step):
        density_matrix = self.system.density_matrix
        if not isinstance(density_matrix, DensityMatrix):
            raise TypeError(f"Density matrix at step {step} is not a DensityMatrix object.")

        purity = np.trace(density_matrix.data @ density_matrix.data)
        self.purity_tracker[step] = purity.real

    def calculate_fidelity(self, step):
        fidelity = state_fidelity(DensityMatrix(self.system.initial_state), self.system.density_matrix)
        self.fidelity_tracker[step] = fidelity

    def plot_results(self):
        fig, axs = plt.subplots(4, 2, figsize=(20, 40))

        # Surplus Accumulation Over Time
        im = axs[0, 0].imshow(self.system.tracker, aspect='auto', cmap='hot')
        axs[0, 0].set_title("Surplus Accumulation Over Time")
        axs[0, 0].set_xlabel("Steps")
        axs[0, 0].set_ylabel("Qubit Index")
        fig.colorbar(im, ax=axs[0, 0], label='Surplus Value')

        # System Entropy Over Time
        axs[0, 1].plot(range(self.params.steps), self.entropy_tracker)
        axs[0, 1].set_title("System Entropy Over Time")
        axs[0, 1].set_xlabel("Steps")
        axs[0, 1].set_ylabel("Average Entropy")

        # Distinction Events Over Time
        im = axs[1, 0].imshow(self.system.distinction_tracker, aspect='auto', cmap='viridis')
        axs[1, 0].set_title("Distinction Events Over Time")
        axs[1, 0].set_xlabel("Steps")
        axs[1, 0].set_ylabel("Qubit Index")
        fig.colorbar(im, ax=axs[1, 0], label='Distinction Value')

        # Antifinity Factor Over Time
        axs[1, 1].plot(range(self.params.steps), self.system.antifinity_tracker)
        axs[1, 1].set_title("Antifinity Factor Over Time")
        axs[1, 1].set_xlabel("Steps")
        axs[1, 1].set_ylabel("Antifinity Value")

        # Quantum State Purity Over Time
        axs[2, 0].plot(range(self.params.steps), self.purity_tracker)
        axs[2, 0].set_title("Quantum State Purity Over Time")
        axs[2, 0].set_xlabel("Steps")
        axs[2, 0].set_ylabel("Purity")

        # Quantum Coherence (L1 Norm) Over Time
        axs[2, 1].plot(range(self.params.steps), self.coherence_tracker)
        axs[2, 1].set_title("Quantum Coherence (L1 Norm) Over Time")
        axs[2, 1].set_xlabel("Steps")
        axs[2, 1].set_ylabel("L1-norm Coherence")

        # Recursion Impact Over Time
        im = axs[3, 0].imshow(self.system.recursion_tracker, aspect='auto', cmap='plasma')
        axs[3, 0].set_title("Recursion Impact Over Time")
        axs[3, 0].set_xlabel("Steps")
        axs[3, 0].set_ylabel("Qubit Index")
        fig.colorbar(im, ax=axs[3, 0], label='Recursion Impact')

        # Relative Entropy of Coherence Over Time
        axs[3, 1].plot(range(self.params.steps), self.relative_coherence_tracker)
        axs[3, 1].set_title("Relative Entropy of Coherence Over Time")
        axs[3, 1].set_xlabel("Steps")
        axs[3, 1].set_ylabel("Relative Entropy of Coherence")

        plt.tight_layout()
        plt.show()

class ParameterOptimization:
    """
    Utilizes Optuna to optimize simulation parameters based on a chosen metric.
    """
    def __init__(self, params, optimization_metric='coherence', n_trials=100, mutual_info_weight=1.0):
        self.params = params
        self.optimization_metric = optimization_metric
        self.n_trials = n_trials
        self.mutual_info_weight = mutual_info_weight
        self.study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        self.best_simulation = None
        self.best_params = None

    def objective(self, trial):
        # Suggest parameters to be optimized
        trial_params = {
            'sinusoidal_scale': trial.suggest_float('sinusoidal_scale', 1.0, 30.0),
            'harmonic_scale': trial.suggest_float('harmonic_scale', 0.1, 10.0),
            'cubic_scale': trial.suggest_float('cubic_scale', 0.001, 7.0),
            'exponential_scale': trial.suggest_float('exponential_scale', 0.1, 100.0),
            'distinction_epsilon': trial.suggest_float('distinction_epsilon', 1e-5, 0.1, log=True),
            'max_surplus_angle': trial.suggest_float('max_surplus_angle', np.pi/8, np.pi),
            'feedback_scale': trial.suggest_float('feedback_scale', 0.1, 10.0, log=True),
            'memory_decay': trial.suggest_float('memory_decay', 0.01, 0.99),
            'surplus_distribution_std': trial.suggest_float('surplus_distribution_std', 0.01, 20.0, log=True),
            'distinction_variance_reduction': trial.suggest_float('distinction_variance_reduction', 0.01, 0.99),
            'recursion_depth': trial.suggest_int('recursion_depth', 50, 500),
            'antifinity_factor': trial.suggest_float('antifinity_factor', 1e-4, 0.01, log=True),
        }

        # Update simulation parameters
        for key, value in trial_params.items():
            setattr(self.params, key, value)

        # Run simulation
        sim = SimulationManager(self.params)
        sim.run_simulation()

        # Calculate metrics
        fidelity = sim.fidelity_tracker[-1]
        coherence = sim.coherence_tracker[-1]
        avg_mutual_info = sim.mutual_info.get_average_mutual_information()
        avg_mutual_info_value = np.mean(avg_mutual_info) if avg_mutual_info is not None else 0

        # Composite objective
        composite_metric = coherence + self.mutual_info_weight * avg_mutual_info_value
        return composite_metric

    def run_optimization(self):
        self.study.optimize(self.objective, n_trials=self.n_trials, n_jobs=1)

    def get_best_simulation(self):
        self.best_params = self.study.best_params
        logger.info(f"Best parameters found: {self.best_params}")

        # Apply best parameters to simulation
        for key, value in self.best_params.items():
            setattr(self.params, key, value)

        # Run the best simulation with these parameters
        best_sim = SimulationManager(self.params)
        best_sim.run_simulation()
        self.best_simulation = best_sim

    def plot_best_simulation(self):
        if self.best_simulation:
            self.best_simulation.plot_results()
            self.best_simulation.print_statistics()

    def plot_parameter_importance(self):
        # Plot parameter importances using Optuna's visualization tools
        try:
            import optuna.visualization as vis
            vis.plot_param_importances(self.study).show()
        except ImportError:
            logger.warning("Optuna visualization modules not found. Install them with pip install optuna[visualization]")

def create_standard_noise_model(decoherence_rate, noise_type='combined'):
    """
    Creates a standard noise model based on specified decoherence rates and noise types.

    Parameters:
        decoherence_rate (float): Rate of decoherence.
        noise_type (str): Type of noise ('phase', 'amplitude', 'depolarizing', 'combined', or 'none').

    Returns:
        NoiseModel: Configured noise model.
    """
    noise_model = NoiseModel()

    # Define errors based on noise type
    if noise_type == 'phase':
        error_1q = phase_damping_error(decoherence_rate)
    elif noise_type == 'amplitude':
        error_1q = amplitude_damping_error(decoherence_rate)
    elif noise_type == 'depolarizing':
        error_1q = depolarizing_error(decoherence_rate, 1)
    elif noise_type == 'combined':
        amp_error = amplitude_damping_error(decoherence_rate)
        phase_error = phase_damping_error(decoherence_rate)
        error_1q = amp_error.compose(phase_error)
    elif noise_type == 'none':
        error_1q = None
    else:
        raise ValueError(f"Unknown noise type: {noise_type}")

    if error_1q is not None:
        # Apply errors to all single-qubit gates
        noise_model.add_all_qubit_quantum_error(error_1q, ['u1', 'u2', 'u3', 'rx', 'ry', 'rz', 'id'])

    # Define two-qubit errors (e.g., depolarizing noise)
    error_2q = depolarizing_error(decoherence_rate, 2)
    noise_model.add_all_qubit_quantum_error(error_2q, ['cx', 'cz'])

    return noise_model

def main(use_optuna=True):
    """
    Main function to execute simulations and optimizations.
    """
    # Define noise parameters
    decoherence_rate = 0.05
    noise_type = 'combined'
    noise_model = create_standard_noise_model(decoherence_rate, noise_type=noise_type)
    basis_gates = ['u1', 'u2', 'u3', 'rx', 'ry', 'rz', 'cx', 'id']
    coupling_map = None

    # Create default parameters
    default_params = SimulationParameters(
        steps=500,
        num_qubits=4,
        noise_model=noise_model,
        basis_gates=basis_gates,
        coupling_map=coupling_map,
        enable_plotting=False
    )

    if use_optuna:
        # Run Optuna optimization
        optimizer = ParameterOptimization(
            params=default_params,
            optimization_metric='coherence',
            n_trials=50,
            mutual_info_weight=1.0
        )
        logger.info("Starting parameter optimization using Optuna...")
        optimizer.run_optimization()
        logger.info("Optimization completed.")
        optimizer.get_best_simulation()
        optimizer.plot_best_simulation()
        optimizer.plot_parameter_importance()

if __name__ == "__main__":
    main(use_optuna=True)

[NbConvertApp] WARNING | pattern 'Series_4.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Ex